# Grand canonical ensemble transition-matrix Monte Carlo

In this example, [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) methods are employed for a small [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) range from 0 to 5 particles.
To begin, the system is initialized with the minimum number of particles by setting [Metropolis](../../monte_carlo/doc/Metropolis_arguments.rst) acceptance [Criteria](../../monte_carlo/doc/Criteria_arguments.rst) with favorable conditions for adding particles.
The [Metropolis](../../monte_carlo/doc/Metropolis_arguments.rst) criteria are then replaced with [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst).
Typical analysis from the previous tutorials are added, and we add [Checkpoint](../../utils/doc/Checkpoint_arguments.rst) files, [CriteriaUpdater](../../steppers/doc/CriteriaUpdater_arguments.rst), [CriteriaWriter](../../steppers/doc/CriteriaWriter_arguments.rst), and average [Energy](../../steppers/doc/Energy_arguments.rst) of a given [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst).
Finally, the simulation is run until the requested convergence criteria for the [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) algorithm are [complete](../../monte_carlo/doc/Run_arguments.rst).

A small [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) range allows the simulation to run quickly with ample sampling, and thus it is an ideal starting point to test the simulations. To begin, read the previous [NIST SRSW](https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website) values [for LJ](https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm) from file for comparison.

In [1]:
import pandas as pd
import numpy as np

srsw_file = "../test/data/stat150.csv"
fh=pd.read_csv(srsw_file)[:6]
fh['lnPI'] -= np.log(sum(np.exp(fh['lnPI'])))   # renormalize
fh['delta_ln_prob']=fh['lnPI'].diff()
fh

,N,energy,lnPI,energystd,lnPIstd,delta_ln_prob
0,0,-2.312265e-10,-18.707570,6.689238e-10,0.037092,NaN
1,1,-6.057402e-04,-14.037373,6.709198e-10,0.036838,4.670197
2,2,-3.057422e-02,-10.050312,9.649147e-06,0.036964,3.987061
3,3,-8.992832e-02,-6.458921,1.387472e-04,0.037746,3.591391
4,4,-1.784571e-01,-3.145637,3.315245e-05,0.038097,3.313283
5,5,-2.961920e-01,-0.045677,1.348791e-05,0.038458,3.099960


Run a short [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) simulation to compare with these [NIST SRSW](https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm) values.

In [2]:
script="""
MonteCarlo
Configuration cubic_side_length=8 particle_type=lj:/feasst/particle/lj_new.txt
Potential Model=LennardJones
Potential VisitModel=LongRangeCorrections
ThermoParams beta={beta} chemical_potential=-2.352321
FlatHistogram Macrostate=MacrostateNumParticles width=1 min=0 max=5 \
              Bias=TransitionMatrix min_sweeps=50
TrialTranslate weight=0.25 tunable_param=1
TrialTransfer weight=1 particle_type=lj
Let [write]=trials_per_write=1e5 output_file=lj
Log [write].txt
Movie [write].xyz
CheckEnergy trials_per_update=1e5 decimal_places=8
Tune
CriteriaUpdater trials_per_update=1e5
CriteriaWriter [write]_fh.txt
Energy [write]_en.txt multistate=true
Checkpoint checkpoint_file=checkpoint.fst num_hours=0.0001
Run until=complete
""".format(beta=1./1.5)

def run(script):
    with open('script1.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("../../../build/bin/fst < script1.txt > script1.log", shell=True, executable='/bin/bash')
    with open('script1.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# Usage: /Users/user/feasst/build/bin/fst < file.txt
FEASST version 0.25.13
MonteCarlo
Configuration cubic_side_length=8 particle_type=lj:/feasst/particle/lj_new.txt
Potential Model=LennardJones
Potential VisitModel=LongRangeCorrections
ThermoParams beta=0.6666666666666666 chemical_potential=-2.352321
FlatHistogram Bias=TransitionMatrix Macrostate=MacrostateNumParticles max=5 min=0 min_sweeps=50 width=1
TrialTranslate tunable_param=1 weight=0.25
TrialTransfer particle_type=lj weight=1
Log output_file=lj.txt trials_per_write=1e5
Movie output_file=lj.xyz trials_per_write=1e5
CheckEnergy decimal_places=8 trials_per_update=1e5
Tune
CriteriaUpdater trials_per_update=1e5
CriteriaWriter output_file=lj_fh.txt trials_per_write=1e5
Energy multistate=true output_file=lj_en.txt trials_per_write=1e5
Checkpoint checkpoint_file=checkpoint.fst num_hours=0.0001
Run until=complete
# Initializing random number generator with seed: 1749408693
 
 exit: 0


In [3]:
import pandas as pd

"""Compare the free energies and potential energies with the NIST SRSW
https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
"""
fh2=pd.read_csv('lj_fh.txt', comment='#')
en=pd.read_csv('lj_en.txt')
print(fh2[['delta_ln_prob', 'delta_ln_prob_stdev']])
print('energy', en[['average', 'block_stdev']])
for macro in range(1, len(fh2)):
    assert np.abs(fh["delta_ln_prob"][macro] - fh2["delta_ln_prob"][macro]) < 4*fh2["delta_ln_prob_stdev"][macro]
    assert np.abs(fh['energy'][macro] - en['average'][macro]) < 4*(fh["energystd"][macro]**2 + en['block_stdev'][macro]**2)**(1./2.)

   delta_ln_prob  delta_ln_prob_stdev
0            NaN             0.000000
1        4.66914             0.001645
2        3.98803             0.002257
3        3.59332             0.001912
4        3.31309             0.001980
5        3.10125             0.001972
energy         average   block_stdev
0  1.317466e-16  1.953789e-16
1 -6.057400e-04  7.702399e-10
2 -3.047790e-02  2.260309e-04
3 -9.005908e-02  4.629765e-04
4 -1.788966e-01  7.342915e-04
5 -2.965805e-01  1.231712e-03


A number of files should also have been created.
If the [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) method is sampling perfectly, the simulation performs a random walk along the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst).
For larger ranges of [Macrostates](../../flat_histogram/doc/Macrostate_arguments.rst), or for more difficult sampling cases, monitoring the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) can help you determine what conditions are preventing convergence.
For example, the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) as a function of the number of attempts may look like the following:

In [4]:
pd.read_csv("lj.txt", header=0).dropna(axis='columns')

,volume,num_particles_lj,beta,state,energy,LennardJones,LongRangeCorrections,BondTwoBody,BondThreeBody,BondFourBody,trial,TrialTranslate,tunable,TrialAdd,TrialRemove
0,512,5,0.666667,5,-5.615610e-02,-4.101260e-02,-1.514350e-02,0,0,0,100000,0.927751,2.83079,0.047373,0.046741
1,512,0,0.666667,0,-3.669700e-15,-3.330670e-16,-1.517880e-18,0,0,0,200000,0.944879,2.83079,0.256928,0.346506
2,512,2,0.666667,2,-2.422960e-03,1.682680e-16,-2.422960e-03,0,0,0,300000,0.950235,2.83079,0.441354,0.570366
3,512,4,0.666667,4,1.532020e-01,1.628940e-01,-9.691840e-03,0,0,0,400000,0.953083,2.83079,0.534310,0.676512
4,512,4,0.666667,4,-9.691840e-03,3.108620e-15,-9.691840e-03,0,0,0,500000,0.954784,2.83079,0.589223,0.737878
5,512,1,0.666667,1,-6.057400e-04,-3.747000e-15,-6.057400e-04,0,0,0,600000,0.956285,2.83079,0.625951,0.778506
6,512,1,0.666667,1,-6.057400e-04,1.304510e-15,-6.057400e-04,0,0,0,700000,0.957065,2.83079,0.652960,0.807427
7,512,0,0.666667,0,-1.987020e-15,-1.935950e-15,-1.517880e-18,0,0,0,800000,0.957227,2.83079,0.672873,0.828548
8,512,5,0.666667,5,-3.277240e-01,-3.125800e-01,-1.514350e-02,0,0,0,900000,0.957637,2.83079,0.688387,0.844791
9,512,1,0.666667,1,-6.057400e-04,5.630910e-15,-6.057400e-04,0,0,0,1000000,0.957736,2.83079,0.700375,0.857707


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!